<div class="alert alert-success" role="alert">
<h1> Project Title: Development of Face Recognition System for Attendance using Python</h1>

<h2>Submitted by: Sachin Maruti Mundhe</h2>
</div>

# <font color= coral>Objective:
## Develop a system that uses face recognition to automate attendance logging in real-time.</font>

## Step 1: Set Up the Environment
Start by setting up the necessary libraries

In [ ]:
!pip install opencv-python
!pip install face_recognition
!pip install pandas


### A. Face Detection using OpenCV
This code captures a live video feed and detects faces in real-time.

In [1]:
import cv2

# Initialize webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Converting the frame to grayscale (for better performance in face detection)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Use OpenCV's pre-trained face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face Detection', frame)

    # Exit the loop with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
video_capture.release()
cv2.destroyAllWindows()


### B. Face Recognition using face_recognition library
This code recognizes faces and logs attendance.

## Step 2: Load Images for Recognition
First, load and encode images of people whose attendance needs to be recorded. 

In [ ]:
# Example of loading and encoding images
def load_and_encode_image(image_path):
    image = face_recognition.load_image_file(image_path)
    image_encoding = face_recognition.face_encodings(image)[0]
    return image_encoding

# Loading my reference images (known faces)
known_face_encodings = [
    load_and_encode_image(r"C:\Users\Sachin's World\Documents\Research_School\Mine_Face_Recognition\Training_images\Sachin.jpg"),  # Path to your image
    load_and_encode_image(r"C:\Users\Sachin's World\Documents\Research_School\Mine_Face_Recognition\Training_images\Modiji.png"),
]
known_face_names = ['Sachin', 'Modiji']  # Names of the individuals


## Step 3: Start Video Stream and Detect Faces
I have implemented the real-time face detection using OpenCV. By Using the webcam feed to capture video.

In [ ]:
# Opening the video stream from webcam
video_capture = cv2.VideoCapture(0)

# Initializing variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Capturing a single frame
    ret, frame = video_capture.read()
    
    # Resizing frame to improve speed
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    # Converting the image from BGR (OpenCV default) to RGB
    rgb_small_frame = small_frame[:, :, ::-1]
    
    # Processing every other frame for speed improvement
    if process_this_frame:
        # Finding face locations and face encodings in the current frame
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            
            # If a match is found in known_face_encodings
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
            
            face_names.append(name)

    process_this_frame = not process_this_frame

    # Displaying the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scaling back up since the frame was resized
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Drawing a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Labeling the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Displaying the resulting image
    cv2.imshow('Video', frame)

    # Breaking from loop with 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Releasing the webcam
video_capture.release()
cv2.destroyAllWindows()


## Step 4: Marking Attendance in CSV File
When a face is recognized, I have loged the attendance with the current date and time.

In [ ]:
# Created a function to mark attendance
def mark_attendance(name):
    # Reading the existing attendance CSV (or create a new one)
    try:
        attendance_data = pd.read_csv('attendance_logs.csv')
    except FileNotFoundError:
        attendance_data = pd.DataFrame(columns=['Name', 'Date', 'Time'])

    now = datetime.now()
    date_str = now.strftime('%Y-%m-%d')
    time_str = now.strftime('%H:%M:%S')

    # Checking if the person is already marked for the day
    if not ((attendance_data['Name'] == name) & (attendance_data['Date'] == date_str)).any():
        attendance_data = attendance_data.append({'Name': name, 'Date': date_str, 'Time': time_str}, ignore_index=True)
        attendance_data.to_csv('attendance.csv', index=False)
        print(f'{name} marked as present at {time_str}')


I have Added this functionality into the video stream loop to log attendance when a face is recognized.

In [ ]:
# Within the face recognition loop, mark attendance
for (top, right, bottom, left), name in zip(face_locations, face_names):
    # Mark attendance only if face is recognized
    if name != "Unknown":
        mark_attendance(name)


## Step 5: Report Generation
Exported the attendance record as a CSV file.

In [ ]:
# Load the attendance data and display it
attendance_data = pd.read_csv('attendance_logs.csv')
print(attendance_data)


# Step 6: Optional - Building a Face Recognition Using Flask
I have built in depth a  Flask UI :

In [ ]:
pip install flask opencv-python-headless pandas face-recognition


In [ ]:
from flask import Flask, render_template
import cv2
import face_recognition
import pandas as pd
from datetime import datetime
import random

# Initializing Flask app
app = Flask(__name__)

# List of known faces and corresponding names
known_face_encodings = []
known_face_names = ["Sachin", "Modiji"]  # Add any other names if needed

# Loading reference images and encode them
def load_and_encode_image(image_path):
    image = face_recognition.load_image_file(image_path)
    image_encoding = face_recognition.face_encodings(image)[0]
    return image_encoding

# Replace 'sachin.jpg' and 'modiji.jpg' with actual image paths
known_face_encodings = [
    load_and_encode_image("C:\Users\Sachin's World\Documents\Research_School\Mine_Face_Recognition\Training_images\Sachin.jpg"),
    load_and_encode_image("C:\Users\Sachin's World\Documents\Research_School\Mine_Face_Recognition\Training_images\Modiji.png")
]

# Initialize attendance DataFrame
attendance_data = pd.DataFrame(columns=['Name', 'Date', 'Time'])

# Marking attendance
def mark_attendance(name):
    global attendance_data
    now = datetime.now()
    date_str = now.strftime('%Y-%m-%d')
    time_str = now.strftime('%H:%M:%S')
    
    # Check if the person is already marked for today
    if not ((attendance_data['Name'] == name) & (attendance_data['Date'] == date_str)).any():
        attendance_data = pd.concat([attendance_data, pd.DataFrame({'Name': [name], 'Date': [date_str], 'Time': [time_str]})], ignore_index=True)
        attendance_data.to_csv('attendance_logs.csv', index=False)
        print(f'{name} marked as present at {time_str}')

# Route for the homepage that shows the attendance logs
@app.route('/')
def home():
    # Load attendance data from CSV
    data = pd.read_csv('attendance_logs.csv')
    return render_template('attendance_logs.html', tables=[data.to_html(classes='table table-striped')], titles=['Attendance Logs'])

# Route to capture and process face recognition
@app.route('/recognize')
def recognize():
    # Open webcam and capture video
    video_capture = cv2.VideoCapture(0)
    
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        # Resizing the frame for faster processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Detecting faces in the frame
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
                mark_attendance(name)
        
        # Displaying the frame with face detection (optional)
        for (top, right, bottom, left) in face_locations:
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
            
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Showing the result
        cv2.imshow('Face Recognition', frame)

        # Breaking the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Releasing webcam and close window
    video_capture.release()
    cv2.destroyAllWindows()
    
    return "Face recognition complete. Check attendance logs."

if __name__ == '__main__':
    app.run(debug=True)


## Explanation of the Code:

### 1 Flask Setup:

The Flask app serves two routes:
/: Displays the attendance logs.
/recognize: Triggers face recognition using the webcam.

### 2 Face Recognition:
I have loaded and encode reference images of "Sachin" and "Modiji" using the face_recognition library.
The recognize route captures live video from the webcam, detects faces, and compares them to the reference images.

### 3 Mark Attendance:
The mark_attendance function records the recognized person's name, date, and time in a CSV file (attendance_logs.csv).

### 4 Rendering Attendance Logs:

The / route reads from the attendance_logs.csv and renders the data into an HTML table using the pandas.to_html() method.